In [2]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('users_behavior.csv')
df.sample(20)


,calls,minutes,messages,mb_used,is_ultra
576,72.0,498.89,71.0,17450.90,1
1750,28.0,169.93,49.0,21232.14,0
2204,5.0,26.07,106.0,29923.23,1
1334,63.0,457.13,56.0,16211.97,1
1440,46.0,278.10,0.0,17288.02,0
402,64.0,544.95,30.0,17090.92,0
1341,51.0,363.53,29.0,22843.02,1
817,24.0,153.32,91.0,23122.08,0
1351,0.0,0.00,9.0,2923.50,1
1404,32.0,187.52,47.0,19373.91,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Empezaremos con definir las caracteristicas y el objetivo, empezando con una logisticregression y comparar su exactitud al umbral de 0.75.

In [5]:
features = df[['calls', 'minutes', 'messages', 'mb_used']] 
target = df['is_ultra'] 

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25,random_state=12345)


Procedo a usar el modelo LogisticRegresion, para empezar a comparar modelos.

In [6]:
model = LogisticRegression(random_state=54321, solver='liblinear') # inicializa el constructor de regresión logística con los parámetros random_state=54321 y solver='liblinear'
model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
score_train = model.score(features_train, target_train) # calcula la puntuación de accuracy en el conjunto de entrenamiento
score_valid = model.score(features_valid, target_valid) # calcula la puntuación de accuracy en el conjunto de validación

print("Accuracy del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Accuracy del modelo de regresión logística en el conjunto de validación:", score_valid)

Accuracy del modelo de regresión logística en el conjunto de entrenamiento: 0.74149377593361
Accuracy del modelo de regresión logística en el conjunto de validación: 0.753731343283582


Dados los resultados podemos observar que en comparacion al decisiontree y al randomforest, logisticregression tiene una exactitud, mas baja apenas apegandose al umbral fijado.

Haremos un decisiontreeclassifier porque es una tarea de clasificacion.

In [7]:
for depth in range(1,6):  # selecciona el rango del hiperparámetro
    # < escribe tu código aquí >
    model=DecisionTreeClassifier(max_depth=depth,random_state=54321)
    model.fit(features_train,target_train)
    test_p=model.predict(features_valid)
    acc=accuracy_score(target_valid,test_p)
    print("Exactitud de max_depth igual a", depth)
    print("Conjunto de entrenamiento:", acc) # calcula la puntuación de accuracy en el conjunto de entrenamiento
    print("Conjunto de prueba:", accuracy_score(target_valid, test_p)) # calcula la puntuación de accuracy en el conjunto de prueba
    print()

Exactitud de max_depth igual a 1
Conjunto de entrenamiento: 0.75
Conjunto de prueba: 0.75

Exactitud de max_depth igual a 2
Conjunto de entrenamiento: 0.7835820895522388
Conjunto de prueba: 0.7835820895522388

Exactitud de max_depth igual a 3
Conjunto de entrenamiento: 0.7885572139303483
Conjunto de prueba: 0.7885572139303483

Exactitud de max_depth igual a 4
Conjunto de entrenamiento: 0.7810945273631841
Conjunto de prueba: 0.7810945273631841

Exactitud de max_depth igual a 5
Conjunto de entrenamiento: 0.7810945273631841
Conjunto de prueba: 0.7810945273631841



Siguiente de lo anterior procedemos a crear un bosque aleatorio

In [9]:
best_score = 0
best_est = 0
for est in range(1, 11): # selecciona el rango del hiperparámetro
    model = RandomForestClassifier(random_state=54321, n_estimators=est) # configura el número de árboles
    model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
    score = model.score(features_valid, target_valid) # calcula la puntuación de accuracy en el conjunto de validación
    if score > best_score:
        best_score = score# guarda la mejor puntuación de accuracy en el conjunto de validación
        best_est = est # guarda el número de estimadores que corresponden a la mejor puntuación de exactitud

print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))

La exactitud del mejor modelo en el conjunto de validación (n_estimators = 4): 0.7885572139303483


De los 3 modelos el mejor ya que se requiere mas exactitud en este problema es el randomforestclassifier, procedo a dejar el modelo final.

In [10]:
final_model = RandomForestClassifier(random_state=54321, n_estimators=best_est)

final_model.fit(features_train, target_train)

final_validation_score = final_model.score(features_valid, target_valid)
print("La exactitud del modelo final en el conjunto de validación:", final_validation_score)


La exactitud del modelo final en el conjunto de validación: 0.7885572139303483


Ahora haremos una prueba de cordura con un dummyclassifier que sirve para comparar el rendimiento de modelos mas complejos.

In [11]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy="most_frequent", random_state=12345)
dummy.fit(features_train, target_train)
dummy_predictions = dummy.predict(features_valid)
dummy_accuracy = accuracy_score(target_valid, dummy_predictions)
print("Exactitud del DummyClassifier (baseline):", dummy_accuracy)


Exactitud del DummyClassifier (baseline): 0.7002487562189055


Dado que mi modelo tiene resultados con mejor exactitud que la prueba de cordura hecha, significa que los patrones que aprendio son utiles, por lo tanto la seleccion de modelo fue acertada.

Conclusion
Dado que el modelo que mas exactitud tiene entre el decisiontree y el randomforest, es el randomforest, se utilizara para escoger el plan correcto para cada usuario basado en las caracteristicas analizadas en el modelo.